In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from datasets import load_dataset, DatasetDict

cache_dir = "E:\\mp3_processing"

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_16_1", "lg", split="train+validation", use_auth_token=True, cache_dir=cache_dir)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_16_1", "lg", split="test", use_auth_token=True, cache_dir=cache_dir)

e:\Masters\Masters Project\Luganda MP3\Code_Speech2Text\.venv\lib\site-packages\datasets\load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
e:\Masters\Masters Project\Luganda MP3\Code_Speech2Text\.venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
e:\Masters\Masters Project\Luganda MP3\Code_Speech2Text\.venv\lib\site-packages\datasets\load.py:2516: FutureWarning: 'use

In [41]:
from transformers import WhisperForConditionalGeneration,WhisperProcessor

model_checkpoint = "./whisper-small-Swa/checkpoint-4000"
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="swahili", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
import torch
import librosa

def compare_transcriptions(dataset, idx, processor, model):
    audio_file = dataset[idx]["audio"]["path"]
    correct_text = dataset[idx]["sentence"]

    # Load and preprocess the audio
    audio_array, sampling_rate = librosa.load(audio_file, sr=16000)
    input_features = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features

    # Generate the model's transcription
    with torch.no_grad():
        generated_ids = model.generate(input_features)
    model_transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Compare the model's transcription with the correct text
    print("Correct Transcription:", correct_text)
    print("Model Transcription:", model_transcription)

In [43]:
compare_transcriptions(common_voice["test"], 10, processor, model)

Correct Transcription: Bonna baabatizibwa eri Musa mu kire ne mu nnyanja.
Model Transcription: Bannaba batizibwa eri mmosa, mukire ne mu nnyanja.
